# Prediction and Data Post-processing

This notebook combines the subimages into image of original size. It uses X_train data to do the experiment and visualize some of the image to make sure the function works properly.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 

import numpy as np
import cv2
import glob
from math import ceil

from keras.models import Model
from keras.layers import Input,merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D,Dropout,Conv2DTranspose
from keras.layers.normalization import BatchNormalization as bn
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers 
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


//anaconda/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
def load_test():
    """This function takes in test.npy file which is generated in preprocessing.py by Raun
    The size of original np array is (221,100,image_size_x,image_size_y)
    It flatten the array into size (22100,image_size_x,image_size_y)"""
    file_num = len(glob.glob("../X_test_11*.npy"))
    train_set = []
    for num in range(file_num):
        x_file_name = "../X_test_" + str(num) + ".npy"
        x = np.load(x_file_name)
        for array in x:
            train_set.append(array)
    return train_set

In [3]:
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


# negative dice loss since we want the network to minimize it more
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


In [4]:
def slice_test_image(image):
    """This function takes in a image with size (image_size_x,image_size_y)
    It slices the input image into small subimages with size (128*128)
    It also records the number of slices made and if there is overlapping.
    if the image size is not divisible by 128, the last row/col of images overlap with the images from previous row/col"""
    new_images = []
    shape = image.shape
    x = ceil(shape[0]/128.)
    y = ceil(shape[1]/128.)
    x_last = shape[0] % 128
    y_last = shape[1] % 128
    for i in range(x):
        for j in range(y):
            if i == x-1:
                image_x = image[shape[0]-128:,:]
            else:
                image_x = image[(i)*128:(i+1)*128,:]
            if j == y-1:
                new_image = image_x[:,shape[1]-128:]
            else:
                new_image = image_x[:,(j)*128:(j+1)*128]
            new_images.append(new_image)
    return new_images,[x,y,x_last,y_last]

In [5]:
def preprocess(train_set):
    """This function create set of sliced images and masks"""
    subimage_set = []
    size_set = []
    size = len(train_set)
    for i in range(size):
        if i % 5000 == 0:
            print("finished slicing ", i, "images")
        image = train_set[i]
        images,image_shape = slice_test_image(image)
        for i in images:
            subimage_set.append(i)
        size_set.append(image_shape)
        
    return np.array(subimage_set),size_set

In [7]:
test_set = load_test()

In [8]:
x_test,image_shapes = preprocess(test_set)

finished slicing  0 images


In [9]:
def combine_predictions(prediction,image_shapes):
    """This function takes in the prediction generated by unet.
        Input shape (num_of_subimage,image_size_x,image_size_y)
        Output is a set of images which are combined from subimages according to the original size """
    image_index = 0
    image_set = []
    for shape in image_shapes:        
        x,y,x_last,y_last = shape[0],shape[1],shape[2],shape[3]
        for i in range(x):
            image_x = prediction[image_index]
            for j in range(y):
                if image_index < len(prediction) and j!=0:
                    if i == x-1 and x_last != 0:
                        image_x = image_x[-x_last:,:] 
                        if j == y-1 and y_last != 0:
                            image_x = np.concatenate((image_x,prediction[image_index][-x_last:,-y_last:]),axis=1)
                        else:
                            image_x = np.concatenate((image_x,prediction[image_index][-x_last:,:]),axis=1)
                    elif j == y-1 and y_last != 0 and i!=x-1:
                        image_x = np.concatenate((image_x,prediction[image_index][:,-y_last:]),axis=1)                        
                    else:
                        image_x = np.concatenate((image_x,prediction[image_index]),axis=1)
                image_index += 1
            if i == 0:
                image = image_x
            else:
                image = np.concatenate((image,image_x),axis = 0)
        image_set.append(image)
    return np.array(image_set)

In [10]:
x_test=x_test/255.0
x_test=x_test.reshape(x_test.shape+(1,))
x_test.shape

(3600, 128, 128, 1)

In [11]:
model = load_model("../model/basic_unet_dsp_p4_round2.h5", custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef})

In [14]:
x_test.shape

(3600, 128, 128, 1)

In [16]:
prediction = model.predict(x_test[:100],batch_size=4)

In [24]:
count = 0
threshold_count = 0 
for i in prediction.flatten():
    count += 1
    if i > 0.01:
        threshold_count += 1
print(count,threshold_count)

1638400 21


In [12]:
np.save()

In [23]:
prediction = prediction.reshape((-1,128,128))

In [ ]:
out = combine_predictions(prediction,image_shapes)

In [ ]:
plt.imshow(out[0],"gray")

In [ ]:
out.shape

In [ ]:
plt.imshow(test_set[0],"gray")